In [26]:
import os
import pandas as pd
import glob
import pandas as pd
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
import re
# from const import QUERY_CUSTOMERINFO,CONNECT_ORACLE
import sys
sys.path.append(os.path.abspath('../'))
from const import  CONNECT_ORACLE


In [20]:
def clean_column_names(df):
    df.columns = df.columns.str.lower()  
    df.columns = df.columns.str.replace(r'\.', '', regex=True)
    df.columns = df.columns.str.replace(r' ', '_', regex=True)  
    df.columns = df.columns.str.replace('customer_id_no|customer_no|customer_id|national_id', 'customer_no', regex=True) 
    df.columns = df.columns.str.replace(r'loan_no', 'contract_no', regex=True)
    df.columns = df.columns.str.replace(r'mobile.*', 'mobile_no', regex=True)
    df.columns = df.columns.str.replace(r'customer_name/surname\(thai\)', 'customer_name', regex=True)  
    return df


In [29]:
input_folder = "../checker/input/assign_input/"

In [30]:
files = glob.glob(os.path.join(input_folder, "*.xlsx")) + glob.glob(os.path.join(input_folder, "*.csv"))
print('files',files)

files ['../checker/input/assign_input\\Template SMS SF+ non assign 17042025 (BASE).xlsx']


In [31]:
if files:
    file_path = files[0]
    if file_path.endswith(".xlsx"):
        assign = pd.read_excel(file_path,sheet_name='Data', dtype={'MobileNo': str})
        print('assignassignassign',assign)
    elif file_path.endswith(".csv"):
        assign = pd.read_csv(file_path,sheet_name='Data', dtype={'MobileNo': str})   
else:
    print("No file found in the folder.")

assignassignassign        No.              Loan  Customer ID No Customer Name/Surname(Thai)  \
0        1  7240770103224216   1520800109680            วรฤทัย เขียวเพชร   
1        2  7241070104039992   3960500135459               อรุณี ทองชาติ   
2        3  7241170104178724   3510600786668             ชวลิต หารุคำจ๋า   
3        4  7240770103221414   3670200265236                  จอม ขุนทอง   
4        5  7240870103638744   1849901429472            ภานุพงษ์ บานเย็น   
...    ...               ...             ...                         ...   
6462  6463  7230470100367012   1420400073671             ธนาพงษ์ ธุระแพง   
6463  6464  7240370102383727   3310101209799                บุญมี วงเขมร   
6464  6465  7240370102359575   1429900157766           ประสิทธิ์ ไชยเสือ   
6465  6466  7230970101234840   1103701092386          อนุสรณ์ พิทักรักษา   
6466  6467  7240470102534019   3660700646107               ยุพา เอี่ยมอบ   

        MobileNo  Status  AR TODAY  4 Digit  เบอร์ติดต่อกลับ  Len  \

In [32]:
conn = jaydebeapi.connect(
        CONNECT_ORACLE.jclassname,
        CONNECT_ORACLE.url,
        CONNECT_ORACLE.jars,
        CONNECT_ORACLE.lib,
        )
cur = conn.cursor() 	


TypeError: Class oracle.jdbc.OracleDriver is not found